In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

import sys
sys.path.append("../")
import mymodule2 as mm
from exp_guitare_configs import *

from data_loader import *

%matplotlib widget

# plt.rc('text', usetex=True)
# plt.rc('font', family='serif')
plt.rcParams["figure.figsize"] = (12,4)

/usr/local/lib/python3.9/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [3]:
deforme_Nta_Npos[0,0]

{'mat_table': 'Metal',
 'pos': 'pos 1',
 'acc': array([1.1224834 , 1.09806847, 1.08808055, ..., 1.23678965, 1.02926277,
        1.07143401]),
 'fs': 51200,
 'temps': array([0.00000000e+00, 1.95312500e-05, 3.90625000e-05, ...,
        4.99994141e+00, 4.99996094e+00, 4.99998047e+00]),
 'mar': array([-0.00732938, -0.0028235 , -0.00390237, ...,  0.00107949,
        -0.00047536,  0.00196797]),
 'FRF': array([444.21885587  +0.j        ,  56.24355362-167.81856495j,
          6.13733125 -92.26578155j, ..., 375.29820045 +58.24782043j,
        497.84103665+120.27247067j, 446.78179365 +57.54509961j]),
 'freq': array([0.00000000e+00, 1.95312500e-01, 3.90625000e-01, ...,
        2.55994141e+04, 2.55996094e+04, 2.55998047e+04])}